In [ ]:
import pandas as pd
import os
import seaborn as sns

In [ ]:
bp = pd.read_csv('/Users/hari/Data/Building_Permits_Issued.csv')


In [ ]:
bp.head()

In [ ]:
building = bp.loc[bp['Permit Type Description'].str.contains('Building')]

In [ ]:
new_building = building.loc[building['Permit Type Description'].str.contains('New')]

In [ ]:
demolition = bp.loc[bp['Permit Type Description'].str.contains('Demolition')]

In [ ]:
res_demolition = demolition.loc[demolition['Permit Subtype Description'].str.contains('Residential')]

In [ ]:
merged = pd.merge(new_building, res_demolition, how='outer')

In [ ]:
#bp.loc[bp['Permit Subtype'].isin(['CAZ01A001', 'CAZ01A002'])]

In [ ]:
merged_sf = merged.loc[merged['Permit Subtype Description'].str.contains('Single Family Residence')]

In [ ]:
merged_sf.head(100)

In [ ]:
new_commercial = ['CAA03R299', 'CAA03R301', 'CAA03R398', 'CAA03R399', 'CAA05R202']

In [ ]:
new_residential = ['CAA01R301', 'CAA02R302', 'CAA03R298', 'CAA03R299', 'CAA03R301', 'CAA03R398', 'CAA03R399', 'CAA04R301', 'CAA05R202', 'CAA08R301', 'CAA10R301', 'CAZ08A001', 'DPLX2CNDO']

In [ ]:
matched = pd.merge(bp.loc[bp['Permit Subtype'].isin(['CAZ01A001', 'CAZ01A002'])], new_building.loc[new_building['Permit Subtype'].isin(new_commercial + new_residential)], on = 'Address')


In [ ]:
#bp['Permit Type Description'].value_counts()

In [ ]:
#bp['Permit Subtype Description'].value_counts()

In [ ]:
commercialdemo = bp.loc[bp['Permit Subtype']== 'CAZ01A001']

In [ ]:
sf_2009 = pd.read_csv('/Users/hari/Data/2009 single family_final_with_sqft.txt', index_col = 'APN')
sf_2013 = pd.read_csv('/Users/hari/Data/2013 single family_final_with_sqft.txt',index_col = 'APN')
sf_2017 = pd.read_csv('/Users/hari/Data/2017 single family_final_with_sqft.txt',index_col = 'APN')

In [ ]:
sf_2017.loc[sf_2017.AddressFullAddress== '605 SUMMIT OAKS CT']

#building = bp.loc[bp['Permit Type Description'].str.contains('Building')]

In [ ]:
print(sf_2009.shape)
sf_2009.head()

In [ ]:
print(sf_2013.shape)
sf_2013.head()

In [ ]:
print(sf_2017.shape)
sf_2017.head()

In [ ]:
sf1 = pd.merge(sf_2009, sf_2013, on = 'APN')
sf1.head()

In [ ]:
sf1.head()

In [ ]:
tax_2016 = pd.read_csv('/Users/hari/Data/2016tax.csv')

In [ ]:
tax_2016.loc[tax_2016.STATE=='TN']